In [1]:
import os
import subprocess
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown
import time
import gradio as gr 

In [2]:
# Load ENV

load_dotenv()

open_ai_api_key = os.getenv("OPENAI_API_KEY")
if open_ai_api_key:
    print(f"Api key exists and begin: {open_ai_api_key[:7]}...")
else:
    print("Api key not found.")

Api key exists and begin: sk-proj...


In [3]:
# iniciaze a openai

openai = OpenAI()

In [4]:
# SET CONSTANS

TRANSCRIPTION_MODEL = 'whisper-1'
SUMMARIZE_MODEL = "gpt-4o-mini"

VIDEO_URL = "https://www.youtube.com/watch?v=toJfrYinzJ8"
AUDIO_FILENAME = 'audio/audio_from_wideo.mp3'

In [5]:
# Download audio from URL VIDEO

def download_audio(url, audio_filename=AUDIO_FILENAME):
    if os.path.exists(audio_filename):
        os.remove(audio_filename)
        # print("Old audio file has been deleted.")
        
    subprocess.run([
        "yt-dlp",
        "-x",                        # only audio
        "--audio-format", "mp3",     # convert to mp3
        "--force-overwrites",        # forve override file
        "-o", audio_filename,        # save file
        url
    ])

    # print(f'Audio has been downloaded as file "{AUDIO_FILENAME}".')

# download_audio(VIDEO_URL, AUDIO_FILENAME)


In [6]:
# Create Transcription from video

def get_transcription(audio_filename=AUDIO_FILENAME, model=TRANSCRIPTION_MODEL, verbose=False):
    with open(audio_filename, 'rb') as audio_file:
        transcription = openai.audio.transcriptions.create(model=model, file=audio_file, response_format="text")
    print(transcription[:200]) if verbose else None
    return transcription

# transcription = get_transcription(audio_filename=AUDIO_FILENAME, model=TRANSCRIPTION_MODEL, verbose=True)

In [7]:
# Set prompts for summarizing

system_message = "You are an assistant that summarizes YouTube videos by highlighting the most important points and events, in clear Markdown format."
user_prompt = f"Below is the transcript of a YouTube video. Please write a summary describing the key moments, main ideas, and highlights in a structured way (using bullet points or short sections)."


In [8]:
def get_summarize(transcription, model=SUMMARIZE_MODEL, system_message=system_message, user_prompt=user_prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"{user_prompt}\n\n{transcription}"}
    ]

    completions = openai.chat.completions.create(
        model=model,
        messages=messages
    )

    return completions.choices[0].message.content

# summarize = get_summarize(model=SUMMARIZE_MODEL, system_message=system_message, user_prompt=user_prompt)

In [9]:
def process_video(url_to_wideo, progress=gr.Progress()):
    
    progress(0, desc="Downloading audio...")
    time.sleep(2)
    download_audio(url_to_wideo)
    
    progress(0.5, desc="Generating transcription...")  # pokazujemy loader
    time.sleep(2)
    transcription = get_transcription()
    
    progress(0.8, desc="Generating summary...")  # pokazujemy loader
    time.sleep(2) 
    summarize = get_summarize(transcription=transcription)

    progress(1.0, desc="Done!")
    return gr.update(visible=False), summarize


def clear_output():
    # ukrywa progress_dummy i czyści Markdown
    return gr.update(visible=False), ""

In [10]:
with gr.Blocks() as demo:
    gr.Markdown(
    """
    # Welcome in Video Summarizer!
    ---
    """)
    
    url_box = gr.Textbox(label="Paste link to video")
    with gr.Row():
        submit_btn = gr.Button("Sumbit", variant="primary")
        clear_btn = gr.Button("Clear") 

    output_md = gr.Markdown(label="Wynik")
    progress_dummy = gr.Textbox(visible=False, show_label=False)

    submit_btn.click(
        fn=lambda x: gr.update(visible=True), 
        inputs=url_box,
        outputs=progress_dummy
    ).then(
        fn=process_video,                      
        inputs=url_box,
        outputs=[progress_dummy, output_md]
    )

    clear_btn.click(
        fn=clear_output,
        inputs=None,
        outputs=[progress_dummy, output_md]
    )

    
if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
